# CNN Training

From our paper on "Explainable Prediction of Acute Myocardial Infarction using Machine Learning and Shapley Values"

In [4]:
pip install keras==2.3.0

     |████████████████████████████████| 377 kB 11.3 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tensorflow==2.2.0

  Using cached tensorflow-2.2.0-cp38-cp38-manylinux2010_x86_64.whl (516.3 MB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
ERROR: Could not install packages due to an OSError: [Errno 39] Directory not empty: 'service'

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import pylab as plt
from keras import optimizers, losses, activations, models, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, Flatten, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.utils import to_categorical
from keras.models import load_model, Sequential
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import os
os.path.abspath('.')

'/userhome/cs/zihaopu/0704/ecg-view-II-machine-learning-master/CNN'

# Loading Data

In [3]:
# Load data
# Import train and test data into dataframes from csv files produced using the data processing code
df_cnn_train = pd.read_csv("/userhome/cs/zihaopu/0704/ecg-view-II-machine-learning-master/data processing/train.csv", header=None)
df_cnn_train = df_cnn_train.sample(frac=1)
df_cnn_test = pd.read_csv("/userhome/cs/zihaopu/0704/ecg-view-II-machine-learning-master/data processing/test.csv", header=None)

In [4]:
type(df_cnn_train)

pandas.core.frame.DataFrame

In [4]:
# Get data from dataframes
y_cnn = np.array(df_cnn_train[11].values).astype(np.int8)
y_cnn=to_categorical(y_cnn)
x_cnn = np.array(df_cnn_train[list(range(11))].values)[..., np.newaxis]
y_cnn_test = np.array(df_cnn_test[11].values).astype(np.int8)
x_cnn_test = np.array(df_cnn_test[list(range(11))].values)[..., np.newaxis]

# Model Definition + Training

In [5]:
# Model definition
def get_model(learning_rate=0.001):
    nclass = 2
    inp = Input(shape=(11, 1))
    
    cnn = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    cnn = Dropout(rate=0.1)(cnn) 
    cnn = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(cnn)
    cnn = Dropout(rate=0.1)(cnn)
    cnn = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(cnn)
    cnn = Dropout(rate=0.1)(cnn)
    cnn = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(cnn)
    cnn = GlobalMaxPool1D()(cnn)
    cnn = Dropout(rate=0.1)(cnn)
    dense_1 = Dense(64, activation=activations.relu, name="dense_1", kernel_regularizer=regularizers.l2(l=0.1))(cnn)
    dense_1 = Dense(16, activation=activations.relu, name="dense_2", kernel_regularizer=regularizers.l2(l=0.1))(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_ecg_view")(dense_1) 
    

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(learning_rate)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model

In [6]:
# Model Training
# Load model and model summary
model = get_model()
model.summary()

# File path to save the model
file_path = "cnn_ecgview.h5"

# Checkpoint the model's weight based on the accuracy of the model
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Set early stopping based on accuracy. It stops after 10 consecutive epochs of no accuracy improvement.
early = EarlyStopping(monitor="val_acc", mode="max", patience=10, verbose=1)

# Reduce learning rate based on accuracy. It reduces the rate after 7 consecutive epochs of no accuracy improvement.
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=7, verbose=2)

callbacks_list = [checkpoint, early, redonplat]

# Train the model, load weights into above file path to save the model
model.fit(x_cnn, y_cnn, epochs=20, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

# The file will be saved in the file_path and can be loaded later using Keras for evaluation

ERROR! Session/line number was not unique in database. History logging moved to new session 126
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 11, 1)             0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 7, 16)             96        
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 16)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 5, 32)             1568      
_________________________________________________________________
dropout_6 (Dropout)          (None, 5, 32)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 3, 64)             6208      
_____________________________